In [1]:
from torchtext.datasets import Multi30k
import torch
import torch.nn as nn
import torch.utils.data
import SentenceLoader

In [2]:
root = './data'
device = 'cuda'
batch_size = 128

train_data = Multi30k(root, split='train')
sentence_loader = SentenceLoader.SentenceLoader(train_data, batch_size=32)

In [3]:
class PositionalEncode(nn.Module):
    def __init__(self, input_shape, device='cuda'):
        super().__init__()
        self.encoding = torch.zeros(input_shape, requires_grad=False, device=device)
        
        i = torch.arange(0, input_shape[1], 2, requires_grad=False, device=device).float().unsqueeze(dim=0)
        pos = torch.arange(0, input_shape[0], 1, requires_grad=False, device=device).float().unsqueeze(dim=1)
        
        self.encoding[:, 0::2] = torch.sin(pos / torch.pow(10000, i / input_shape[1]))
        self.encoding[:, 1::2] = torch.cos(pos / torch.pow(10000, i / input_shape[1]))
        
        return
    
    def forward(self, x):
        
        x = x + self.encoding
        return x

In [9]:
en, ge = next(iter(sentence_loader))
en = en.to(device)
embedding = nn.Embedding(en.shape[1], 512, device=device)
data_en = embedding(en)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [8]:
pos_encode = PositionalEncode(data_en.shape[1:])

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        

torch.Size([32, 512, 512])
